# Final Project
Wil Gaboury and Calvin Hirsch

In [354]:
import pandas as pd
import warnings
import numpy as np

In [355]:
csgo_df = pd.read_csv('./data/csgo.csv')
csgo_df = csgo_df[pd.to_numeric(csgo_df.best_of, errors='coerce').notnull()]
csgo_df

/home/calvin/projects/cmsc320-final/venv/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,date,match_id,event_id,team_1,team_2,best_of,_map,t1_start,t2_start,1_t1,...,21_winner,22_winner,23_winner,24_winner,25_winner,26_winner,27_winner,28_winner,29_winner,30_winner
0,2020-03-01,2339402,4901,G2,Natus Vincere,5,Nuke,t,ct,4350.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-03-01,2339402,4901,G2,Natus Vincere,5,Dust2,ct,t,3900.0,...,2.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,NaN
2,2020-03-01,2339402,4901,G2,Natus Vincere,5,Mirage,t,ct,4150.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-29,2339401,4901,Natus Vincere,Astralis,3,Dust2,t,ct,4150.0,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-29,2339401,4901,Natus Vincere,Astralis,3,Nuke,ct,t,4200.0,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43229,2017-04-04,2309263,2683,FaZe,HellRaisers,1,Cache,t,ct,4250.0,...,2.0,1.0,1.0,1.0,1.0,2.0,NaN,NaN,NaN,NaN
43230,2017-04-04,2309262,2683,Astralis,CLG,1,Train,t,ct,4250.0,...,1.0,1.0,1.0,2.0,1.0,1.0,1.0,NaN,NaN,NaN
43231,2017-04-04,2309261,2683,SK,TYLOO,1,Mirage,ct,t,4350.0,...,1.0,2.0,1.0,1.0,1.0,2.0,1.0,1.0,NaN,NaN
43232,2017-04-04,2309260,2683,Virtus.pro,MVP Project,1,Mirage,t,ct,4150.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [356]:
csgo_df['win_ratio'] = csgo_df.loc[:, '1_winner':'30_winner'].apply(lambda x: len(x[x == 1]) - len(x[x == 2]), axis=1)
csgo_df.loc[:, '1_winner':'30_winner'] = csgo_df.loc[:, '1_winner':'30_winner'].applymap(lambda x: True if x == 1 else False if x == 2 else None)

csgo_df['t1_start_ct'] = csgo_df['t1_start'].apply(lambda x: x == 'ct')
csgo_df = csgo_df.drop(['t1_start', 't2_start'], axis=1)

csgo_df = csgo_df.rename(columns={'_map':'map'})

csgo_df[['t1_start_ct', '1_winner', '30_winner', 'map', 'win_ratio']]

,t1_start_ct,1_winner,30_winner,map,win_ratio
0,False,False,None,Nuke,-12
1,True,False,None,Dust2,-3
2,False,False,None,Mirage,-14
3,False,True,None,Dust2,11
4,True,False,None,Nuke,11
...,...,...,...,...,...
43229,False,False,None,Cache,-6
43230,False,False,None,Train,5
43231,True,False,None,Mirage,4
43232,False,False,None,Mirage,12


In [357]:
maps = ["Nuke", "Dust2", "Mirage", "Train", "Cache", "Cobblestone", "Vertigo", "Inferno", "Overpass"]

classifier_df = csgo_df.copy()
classifier_df = classifier_df.drop(['event_id', 'match_id', 'date', 'best_of', 'team_1', 'team_2'], axis=1)
classifier_df = classifier_df.drop(classifier_df.loc[:, '16_winner': '30_winner'].columns, axis=1)
classifier_df = classifier_df.drop(classifier_df.loc[:, '16_t1': '30_t1'].columns, axis=1)
classifier_df = classifier_df.drop(classifier_df.loc[:, '16_t2': '30_t2'].columns, axis=1)
classifier_df = classifier_df.query("map in list(" + str(maps) + ")")
classifier_df = classifier_df.sample(frac=1).reset_index(drop=True)
label_df = classifier_df["win_ratio"]
classifier_df = classifier_df.drop(['win_ratio'], axis=1)

classifier_df

,map,1_t1,2_t1,3_t1,4_t1,5_t1,6_t1,7_t1,8_t1,9_t1,...,7_winner,8_winner,9_winner,10_winner,11_winner,12_winner,13_winner,14_winner,15_winner,t1_start_ct
0,Train,4300.0,23400.0,25550.0,3050.0,26350.0,3000.0,24800.0,9600.0,27950.0,...,False,False,True,True,True,True,True,True,True,True
1,Mirage,4300.0,14850.0,24750.0,24150.0,25700.0,21750.0,19700.0,4150.0,23000.0,...,False,False,True,False,True,False,False,True,True,False
2,Mirage,4300.0,6700.0,21950.0,13750.0,1200.0,23450.0,9350.0,29850.0,8800.0,...,True,False,False,True,False,True,True,True,True,True
3,Inferno,4200.0,14650.0,19900.0,16150.0,24300.0,22500.0,24100.0,23700.0,24500.0,...,True,True,True,False,True,False,False,False,False,False
4,Dust2,4200.0,22200.0,22250.0,29200.0,1900.0,25000.0,26850.0,20400.0,1000.0,...,False,False,False,False,False,True,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43034,Cache,4100.0,17850.0,10550.0,1400.0,22850.0,27300.0,7350.0,6800.0,24600.0,...,False,False,False,False,False,False,True,False,False,True
43035,Train,3650.0,18000.0,22400.0,23100.0,23300.0,24600.0,25350.0,8550.0,23100.0,...,False,False,True,False,False,True,False,False,False,False
43036,Inferno,4350.0,21000.0,26400.0,16900.0,1300.0,23150.0,6100.0,27350.0,6500.0,...,False,False,False,True,True,True,True,True,True,True
43037,Inferno,4200.0,10100.0,1400.0,26300.0,26500.0,24900.0,25000.0,2200.0,23550.0,...,False,False,True,False,False,False,True,True,True,True


In [358]:
for ind in range(0, len(classifier_df)):
    for map in maps:
        classifier_df.at[ind, "is" + map] = classifier_df.at[ind, "map"] == map

del classifier_df["map"]
        
for i in range(1, 16):
    classifier_df["{}_t1".format(i)] = classifier_df["{}_t1".format(i)].apply(lambda x: x / 10000)
    classifier_df["{}_t2".format(i)] = classifier_df["{}_t2".format(i)].apply(lambda x: x / 10000)

classifier_df.head(14)

,1_t1,2_t1,3_t1,4_t1,5_t1,6_t1,7_t1,8_t1,9_t1,10_t1,...,t1_start_ct,isNuke,isDust2,isMirage,isTrain,isCache,isCobblestone,isVertigo,isInferno,isOverpass
0,0.430,2.340,2.555,0.305,2.635,0.300,2.480,0.960,2.795,2.780,...,True,False,False,False,True,False,False,False,False,False
1,0.430,1.485,2.475,2.415,2.570,2.175,1.970,0.415,2.300,2.480,...,False,False,False,True,False,False,False,False,False,False
2,0.430,0.670,2.195,1.375,0.120,2.345,0.935,2.985,0.880,2.365,...,True,False,False,True,False,False,False,False,False,False
3,0.420,1.465,1.990,1.615,2.430,2.250,2.410,2.370,2.450,2.420,...,False,False,False,False,False,False,False,False,True,False
4,0.420,2.220,2.225,2.920,0.190,2.500,2.685,2.040,0.100,2.255,...,True,False,True,False,False,False,False,False,False,False
5,0.425,2.020,1.830,0.530,2.975,1.215,0.120,2.355,0.505,2.950,...,True,False,False,False,False,True,False,False,False,False
6,0.435,1.980,2.495,2.020,0.100,2.450,2.770,2.880,0.260,2.095,...,True,False,False,False,False,True,False,False,False,False
7,0.435,2.045,2.185,2.405,2.495,2.955,2.985,3.190,3.300,3.185,...,True,False,False,False,False,True,False,False,False,False
8,0.410,0.725,0.140,2.240,0.405,2.515,2.150,0.715,2.400,2.625,...,False,False,False,False,False,True,False,False,False,False
9,0.440,1.930,2.130,2.850,0.675,2.300,0.460,2.980,2.600,0.920,...,True,False,False,True,False,False,False,False,False,False


In [365]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.l1 = nn.Linear(55, 20)
        self.l2 = nn.Linear(20, 5)
        self.l3 = nn.Linear(5, 1)
        #torch.nn.init.xavier_uniform_(self.l1.weight)
        #torch.nn.init.xavier_uniform_(self.l2.weight)
        #torch.nn.init.xavier_uniform_(self.l3.weight)
        
    def forward(self, x):
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x = self.l3(x)
        return x

In [366]:
import torch.optim as optim

c = Classifier()

criterion = nn.BCELoss()
optimizer = optim.SGD(c.parameters(), lr=0.001, momentum=0.9)

In [367]:
import torch.utils.data as data_utils

batch_size = 32
test_size = 2000

train_df = classifier_df.head(-test_size)
train_labels_df = label_df.head(-test_size)
test_df = classifier_df.tail(test_size)
test_labels_df = label_df.tail(test_size)

train_features = torch.tensor(train_df.values.astype(np.float32))
train_labels = torch.tensor(train_labels_df.values.astype(np.float32)).unsqueeze(-1)
train_set = data_utils.TensorDataset(train_features, train_labels)

test_features = torch.tensor(test_df.values.astype(np.float32))
test_labels = torch.tensor(test_labels_df.values.astype(np.float32)).unsqueeze(-1)
test_set = data_utils.TensorDataset(test_features, test_labels)

train_dl = data_utils.DataLoader(train_set, batch_size=batch_size, shuffle=True)
test_dl = data_utils.DataLoader(test_set, batch_size=batch_size, shuffle=True)
len(train_dl)

1283

In [368]:
print_interval = 5
batches_per_epoch = 43030.0 / batch_size

for epoch in range(50):
    running_loss = 0.0
    running_acc = 0.0
    for i, data in enumerate(train_dl, 0):
        inputs, labels = data
        
        optimizer.zero_grad()
        
        outputs = c(inputs)
            
        loss = criterion(outputs.clamp(-16, 16) / 32 + 0.5, labels / 32 + 0.5)
        loss.backward()
        optimizer.step()
                
        running_acc += np.mean(np.sign(outputs.detach().numpy()) == np.sign(labels.detach().numpy()))
        running_loss += loss.item()
        
    if epoch % print_interval == 0:
        print("%d - loss: %.3f, %.2f%% acc" %
              (epoch, running_loss / batches_per_epoch, 100 * running_acc / batches_per_epoch))
        running_loss = 0.0

print('Finished Training')

0 - loss: 0.661, 48.66% acc
5 - loss: 0.605, 68.22% acc
10 - loss: 0.592, 69.47% acc
15 - loss: 0.589, 69.97% acc
20 - loss: 0.588, 70.21% acc
25 - loss: 0.587, 70.28% acc
30 - loss: 0.587, 70.32% acc
35 - loss: 0.587, 70.39% acc
40 - loss: 0.586, 70.38% acc
45 - loss: 0.586, 70.39% acc
Finished Training


In [369]:
correct = 0
dist = 0

for i,data in enumerate(test_dl, 0):
    inputs, labels = data
    
    outputs = c(inputs)
    
    correct += np.sum(np.sign(outputs.detach().numpy()) == np.sign(labels.detach().numpy()))
    dist += np.sum(np.abs(labels.detach().numpy() - outputs.detach().numpy()))

print("Winner Accuracy: {}".format(100 * correct / test_size))
print("Average Distance: {}".format(dist / test_size))

Winner Accuracy: 73.55
Average Distance: 3.838923017501831


In [364]:
np.sign(0)

0